In [ ]:
import requests
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

Uma função para buscar os dados com a API do banco central do brasil BCB

In [ ]:
def buscar_serie_bcb(codigo, nome_coluna, dias_maximos=365*10):
    
    data_final = datetime.today()
    data_inicial = data_final - timedelta(days=dias_maximos)
    
    data_inicial_str = data_inicial.strftime('%d/%m/%Y')
    data_final_str = data_final.strftime('%d/%m/%Y')

    url = (
        f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo}/dados"
        f"?formato=json&dataInicial={data_inicial_str}&dataFinal={data_final_str}"
    )

    resposta = requests.get(url)

    try:
        dados = resposta.json()
        if not isinstance(dados, list):
            raise ValueError(f"Resposta inesperada para o código {codigo}: {dados}")
        
        df = pd.DataFrame(dados)
        df['data'] = pd.to_datetime(df['data'], dayfirst=True)
        df[nome_coluna] = df['valor'].str.replace(',', '.').astype(float)
        return df[['data', nome_coluna]]
    
    except Exception as e:
        print(f"[ERRO] Falha ao coletar série {codigo} - {nome_coluna}: {e}")
        return pd.DataFrame(columns=['data', nome_coluna])


In [ ]:
defino com os códigos quais dados gostaria de coletar

In [ ]:
df_selic = buscar_serie_bcb(11, 'selic')      
df_ipca  = buscar_serie_bcb(433, 'ipca')       
df_dolar = buscar_serie_bcb(1, 'dolar')        

Reuni os dados todos em um dataframe

In [10]:
df = pd.merge(df_selic, df_dolar, on='data', how='outer')
df = pd.merge(df, df_ipca, on='data', how='outer')
df

,data,selic,dolar,ipca
0,2015-04-01,NaN,NaN,0.71
1,2015-04-13,0.047279,3.1011,NaN
2,2015-04-14,0.047279,3.0880,NaN
3,2015-04-15,0.047279,3.0681,NaN
4,2015-04-16,0.047279,3.0249,NaN
...,...,...,...,...
2551,2025-04-03,0.052531,5.6067,NaN
2552,2025-04-04,0.052531,5.7777,NaN
2553,2025-04-07,0.052531,5.8877,NaN
2554,2025-04-08,0.052531,5.9368,NaN


A próxima célula realiza a ordenação dos dados no DataFrame `df` com base na coluna `data` e, em seguida, remove as linhas que possuem valores ausentes (NaN). O resultado é armazenado no DataFrame `df_limpo`, que contém apenas os dados completos e ordenados cronologicamente. Essa etapa é essencial para garantir a consistência e a qualidade dos dados antes de realizar análises ou visualizações.

In [5]:
# Ordena as datas e remove linhas com dados faltantes
df = df.sort_values('data')
df_limpo = df.dropna()

Então realizamos o plot do gráfico 

In [ ]:

# === Visualização com Plotly ===
fig = px.line(df_limpo, x='data', y=['selic', 'dolar', 'ipca'],
              labels={'value': 'Valor', 'variable': 'Indicador'},
              title='Indicadores Econômicos: SELIC, Dólar e IPCA (Brasil)')

fig.update_layout(template='plotly_dark')
fig.show()
